<a href="https://colab.research.google.com/github/YUCAVALCANTE/Tutorials_Linkedin/blob/main/Construindo_%C3%81rvores_de_Decis%C3%A3o_do_Zero_Usando%C2%A0Polars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Instalar dependências
!pip install kagglehub polars pandas scikit-learn numpy time

# Importar as bibliotecas necessárias
import kagglehub
import pandas as pd
import polars as pl
import time

# Baixando o dataset do Kaggle
dataset_path = kagglehub.dataset_download("jainilcoder/online-payment-fraud-detection")

print("Path para os arquivos:", dataset_path)

ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time


100%|██████████| 178M/178M [00:01<00:00, 109MB/s]

Extracting files...


Path para os arquivos: /root/.cache/kagglehub/datasets/jainilcoder/online-payment-fraud-detection/versions/1


In [4]:
import os
print("Arquivos no dataset:", os.listdir(dataset_path))

Arquivos no dataset: ['onlinefraud.csv']


In [5]:
file_path = f"{dataset_path}/onlinefraud.csv"

# Carregando com Pandas
start_time = time.time()
df_pandas = pd.read_csv(file_path)
tempo_pandas = time.time() - start_time

# Carregando com Polars
start_time = time.time()
df_polars = pl.read_csv(file_path)
tempo_polars = time.time() - start_time

# Exibindo os tempos de carregamento
print(f"Tempo de carregamento - Pandas: {tempo_pandas:.4f}s | Polars: {tempo_polars:.4f}s")


Tempo de carregamento - Pandas: 26.1809s | Polars: 3.2676s


In [6]:
# Verificando as primeiras linhas com Pandas
print(df_pandas.head())

# Verificando as primeiras linhas com Polars
print(df_polars.head())


   step      type    amount     nameOrig  oldbalanceOrg  newbalanceOrig  \
0     1   PAYMENT   9839.64  C1231006815       170136.0       160296.36   
1     1   PAYMENT   1864.28  C1666544295        21249.0        19384.72   
2     1  TRANSFER    181.00  C1305486145          181.0            0.00   
3     1  CASH_OUT    181.00   C840083671          181.0            0.00   
4     1   PAYMENT  11668.14  C2048537720        41554.0        29885.86   

      nameDest  oldbalanceDest  newbalanceDest  isFraud  isFlaggedFraud  
0  M1979787155             0.0             0.0        0               0  
1  M2044282225             0.0             0.0        0               0  
2   C553264065             0.0             0.0        1               0  
3    C38997010         21182.0             0.0        1               0  
4  M1230701703             0.0             0.0        0               0  
shape: (5, 11)
┌──────┬──────────┬──────────┬─────────────┬───┬─────────────┬─────────────┬─────────┬────

In [7]:
# Número de linhas e colunas
print(f"Dimensão dos dados - Pandas: {df_pandas.shape}")
print(f"Dimensão dos dados - Polars: {df_polars.shape}")

# Verificando as colunas disponíveis
print("Colunas no dataset:", df_pandas.columns)


Dimensão dos dados - Pandas: (6362620, 11)
Dimensão dos dados - Polars: (6362620, 11)
Colunas no dataset: Index(['step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig',
       'nameDest', 'oldbalanceDest', 'newbalanceDest', 'isFraud',
       'isFlaggedFraud'],
      dtype='object')


In [8]:
# Verificando valores nulos com Pandas
print("Valores nulos no dataset:")
print(df_pandas.isnull().sum())

# Fazendo o mesmo com Polars
print("Valores nulos com Polars:")
print(df_polars.null_count())

Valores nulos no dataset:
step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64
Valores nulos com Polars:
shape: (1, 11)
┌──────┬──────┬────────┬──────────┬───┬────────────────┬────────────────┬─────────┬────────────────┐
│ step ┆ type ┆ amount ┆ nameOrig ┆ … ┆ oldbalanceDest ┆ newbalanceDest ┆ isFraud ┆ isFlaggedFraud │
│ ---  ┆ ---  ┆ ---    ┆ ---      ┆   ┆ ---            ┆ ---            ┆ ---     ┆ ---            │
│ u32  ┆ u32  ┆ u32    ┆ u32      ┆   ┆ u32            ┆ u32            ┆ u32     ┆ u32            │
╞══════╪══════╪════════╪══════════╪═══╪════════════════╪════════════════╪═════════╪════════════════╡
│ 0    ┆ 0    ┆ 0      ┆ 0        ┆ … ┆ 0              ┆ 0              ┆ 0       ┆ 0              │
└──────┴──────┴────────┴──────────┴───┴────────────────┴────────────────┴─────────┴───────────

In [9]:
# Removendo duplicatas no Pandas
df_pandas = df_pandas.drop_duplicates()

# Removendo duplicatas no Polars
df_polars = df_polars.unique()

In [10]:
print(df_polars.schema)  # Confirma os tipos das colunas
print(df_polars.head(5))  # Visualiza os primeiros valores


Schema([('step', Int64), ('type', String), ('amount', Float64), ('nameOrig', String), ('oldbalanceOrg', Float64), ('newbalanceOrig', Float64), ('nameDest', String), ('oldbalanceDest', Float64), ('newbalanceDest', Float64), ('isFraud', Int64), ('isFlaggedFraud', Int64)])
shape: (5, 11)
┌──────┬──────────┬───────────┬─────────────┬───┬─────────────┬─────────────┬─────────┬────────────┐
│ step ┆ type     ┆ amount    ┆ nameOrig    ┆ … ┆ oldbalanceD ┆ newbalanceD ┆ isFraud ┆ isFlaggedF │
│ ---  ┆ ---      ┆ ---       ┆ ---         ┆   ┆ est         ┆ est         ┆ ---     ┆ raud       │
│ i64  ┆ str      ┆ f64       ┆ str         ┆   ┆ ---         ┆ ---         ┆ i64     ┆ ---        │
│      ┆          ┆           ┆             ┆   ┆ f64         ┆ f64         ┆         ┆ i64        │
╞══════╪══════════╪═══════════╪═════════════╪═══╪═════════════╪═════════════╪═════════╪════════════╡
│ 210  ┆ CASH_OUT ┆ 625288.73 ┆ C908224299  ┆ … ┆ 1.6417e6    ┆ 2.2669e6    ┆ 0       ┆ 0          │
│ 258  

In [11]:
import pandas as pd

# Garantindo que 'amount' está no formato float
df_pandas["amount"] = df_pandas["amount"].astype(float)

# O 'step' representa um número de horas passadas. Podemos criar uma referência temporal
base_date = pd.Timestamp("2023-01-01")  # Escolhendo um ponto de partida fictício
df_pandas["step"] = base_date + pd.to_timedelta(df_pandas["step"], unit="h")

# Convertendo colunas categóricas para tipo "category" (mais eficiente para grandes datasets)
df_pandas["type"] = df_pandas["type"].astype("category")
df_pandas["isFraud"] = df_pandas["isFraud"].astype("int8")  # Reduzindo consumo de memória


In [12]:
import polars as pl

# Aplicando conversões no Polars
df_polars = df_polars.with_columns([
    pl.col("amount").cast(pl.Float64),  # Garantindo que 'amount' é float
    (pl.lit("2023-01-01").str.strptime(pl.Date, "%Y-%m-%d") + pl.duration(hours=pl.col("step"))).alias("timestamp"),  # Criando um timestamp
    pl.col("step").cast(pl.Int64),  # Mantendo como inteiro para representar horas
    pl.col("type").cast(pl.Categorical),  # Transformando 'type' em categórico para otimização
    pl.col("isFraud").cast(pl.Int8)  # Reduzindo consumo de memória
])


In [13]:
# Criando uma nova variável - Exemplo de feature engineering
df_pandas["valor_suspeito"] = df_pandas["amount"] > df_pandas["amount"].quantile(0.99)

df_polars = df_polars.with_columns(
    (pl.col("amount") > df_polars["amount"].quantile(0.99)).alias("valor_suspeito")
)


In [15]:
# Garantindo que 'step' seja um valor numérico e não datetime
df_pandas["step"] = df_pandas["step"].astype(int)  # Convertendo para inteiro


In [16]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

# Garantindo que 'step' seja numérico
df_pandas["step"] = df_pandas["step"].astype(int)

# Definindo as features e a variável-alvo
features = ["step", "amount", "oldbalanceOrg", "newbalanceOrig"]
target = "isFraud"

# Dividindo os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    df_pandas[features], df_pandas[target], test_size=0.2, random_state=42
)

# Criando e treinando o modelo de árvore de decisão
modelo_sklearn = DecisionTreeClassifier(max_depth=5, random_state=42)
modelo_sklearn.fit(X_train, y_train)

# Fazendo previsões
y_pred = modelo_sklearn.predict(X_test)

# Avaliando o modelo
print("Acurácia:", accuracy_score(y_test, y_pred))
print("Relatório de Classificação:\n", classification_report(y_test, y_pred))


Acurácia: 0.9990986417544974
Relatório de Classificação:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00   1270904
           1       0.74      0.45      0.56      1620

    accuracy                           1.00   1272524
   macro avg       0.87      0.72      0.78   1272524
weighted avg       1.00      1.00      1.00   1272524



In [22]:
import polars as pl

def calcular_gini(df: pl.DataFrame, target: str) -> float:
    """Calcula a impureza de Gini de um conjunto de dados."""

    # Contando as ocorrências de cada classe
    proporcoes = df[target].value_counts()

    # Ajustando para acessar corretamente a contagem de classes
    total = df.shape[0]
    proporcoes = proporcoes.with_columns(
        (pl.col("count") / total).alias("proporcao")  # Agora acessamos diretamente "count"
    )

    # Calculando o índice de Gini
    gini = 1 - (proporcoes["proporcao"] ** 2).sum()
    return gini

# Executando novamente
gini_inicial = calcular_gini(df_polars, "isFraud")
print(f"Gini do dataset original: {gini_inicial:.4f}")


Gini do dataset original: 0.0026


In [24]:
def encontrar_melhor_split(df: pl.DataFrame, target: str, feature: str, num_pontos: int = 10) -> tuple:
    """Encontra o melhor ponto de corte para uma feature baseada no critério de Gini."""

    # Obtendo valores únicos ordenados da feature para testar divisões
    valores_unicos = df.select(pl.col(feature)).unique().sort(feature).to_series().to_list()

    # Se houver muitos valores, selecionamos apenas alguns para otimizar o processo
    if len(valores_unicos) > num_pontos:
        valores_unicos = pl.Series(valores_unicos).sample(num_pontos, shuffle=True).sort().to_list()

    # Testamos cada ponto de corte e encontramos aquele que gera menor Gini
    melhor_gini = float("inf")
    melhor_ponto = None

    for valor in valores_unicos:
        gini_split = calcular_gini_split(df, target, feature, valor)
        if gini_split < melhor_gini:
            melhor_gini = gini_split
            melhor_ponto = valor

    return melhor_ponto, melhor_gini

# Testando para uma feature
melhor_ponto, melhor_gini = encontrar_melhor_split(df_polars, "isFraud", "amount")
print(f"Melhor ponto de corte para 'amount': {melhor_ponto}, com Gini {melhor_gini:.4f}")


Melhor ponto de corte para 'amount': 710240.67, com Gini 0.0026


In [25]:
def dividir_dados(df: pl.DataFrame, feature: str, valor_corte: float) -> tuple:
    """Divide os dados em dois subconjuntos com base em uma feature e um valor de corte."""
    df_esquerda = df.filter(pl.col(feature) <= valor_corte)
    df_direita = df.filter(pl.col(feature) > valor_corte)
    return df_esquerda, df_direita

# Dividindo os dados com base na melhor separação encontrada
df_esquerda, df_direita = dividir_dados(df_polars, "amount", melhor_ponto)

print(f"Tamanho do grupo esquerdo: {df_esquerda.shape[0]}")
print(f"Tamanho do grupo direito: {df_direita.shape[0]}")

Tamanho do grupo esquerdo: 6163020
Tamanho do grupo direito: 199600


In [26]:
class NoArvore:
    """Classe representando um nó da árvore de decisão."""
    def __init__(self, feature=None, valor_corte=None, gini=None, esquerda=None, direita=None, resultado=None):
        self.feature = feature
        self.valor_corte = valor_corte
        self.gini = gini
        self.esquerda = esquerda
        self.direita = direita
        self.resultado = resultado  # Se for um nó folha, armazenamos a classe majoritária


In [27]:
def construir_arvore(df: pl.DataFrame, target: str, profundidade=0, max_profundidade=5, min_amostras=500) -> NoArvore:
    """Função recursiva para construir a árvore de decisão."""

    # Calcula o Gini do nó atual
    gini_atual = calcular_gini(df, target)

    # Critérios de parada
    if profundidade >= max_profundidade or df.shape[0] < min_amostras or gini_atual == 0:
        resultado = df[target].mode()[0]  # Classe mais frequente no nó
        return NoArvore(resultado=resultado)

    # Encontra a melhor divisão
    melhor_feature, melhor_ponto, melhor_gini = None, None, float("inf")
    for feature in ["amount", "oldbalanceOrg", "newbalanceOrig"]:  # Testamos algumas features
        ponto, gini = encontrar_melhor_split(df, target, feature)
        if gini < melhor_gini:
            melhor_gini, melhor_ponto, melhor_feature = gini, ponto, feature

    # Se não encontrou um bom split, retorna um nó folha
    if melhor_feature is None:
        resultado = df[target].mode()[0]
        return NoArvore(resultado=resultado)

    # Divide os dados
    df_esquerda, df_direita = dividir_dados(df, melhor_feature, melhor_ponto)

    # Chama recursivamente para criar os nós filhos
    no_esquerda = construir_arvore(df_esquerda, target, profundidade + 1, max_profundidade, min_amostras)
    no_direita = construir_arvore(df_direita, target, profundidade + 1, max_profundidade, min_amostras)

    return NoArvore(feature=melhor_feature, valor_corte=melhor_ponto, gini=melhor_gini,
                    esquerda=no_esquerda, direita=no_direita)


In [28]:
# Criando a árvore de decisão com profundidade máxima de 5
arvore_decisao = construir_arvore(df_polars, "isFraud", max_profundidade=5, min_amostras=500)

print("Árvore construída com sucesso!")


Árvore construída com sucesso!


In [29]:
def exibir_arvore(no, nivel=0):
    """Imprime a árvore de decisão de forma estruturada."""
    if no is None:
        return

    prefixo = "   " * nivel  # Adiciona indentação para representar os níveis da árvore

    # Se for um nó folha, imprime a classe
    if no.resultado is not None:
        print(f"{prefixo}🌱 Classe: {no.resultado}")
    else:
        print(f"{prefixo}📌 Se {no.feature} <= {no.valor_corte:.2f}:")

        # Recursivamente exibe os nós filhos (esquerda e direita)
        print(f"{prefixo}├── Esquerda:")
        exibir_arvore(no.esquerda, nivel + 1)

        print(f"{prefixo}└── Direita:")
        exibir_arvore(no.direita, nivel + 1)

# Exibindo a árvore gerada
exibir_arvore(arvore_decisao)


📌 Se amount <= 568443.32:
├── Esquerda:
   📌 Se oldbalanceOrg <= 17186.81:
   ├── Esquerda:
      📌 Se amount <= 16611.76:
      ├── Esquerda:
         📌 Se oldbalanceOrg <= 671.10:
         ├── Esquerda:
            📌 Se amount <= 101.84:
            ├── Esquerda:
               🌱 Classe: 0
            └── Direita:
               🌱 Classe: 0
         └── Direita:
            📌 Se newbalanceOrig <= 605.42:
            ├── Esquerda:
               🌱 Classe: 0
            └── Direita:
               🌱 Classe: 0
      └── Direita:
         📌 Se amount <= 18291.55:
         ├── Esquerda:
            📌 Se oldbalanceOrg <= 13346.00:
            ├── Esquerda:
               🌱 Classe: 0
            └── Direita:
               🌱 Classe: 0
         └── Direita:
            📌 Se oldbalanceOrg <= 6843.40:
            ├── Esquerda:
               🌱 Classe: 0
            └── Direita:
               🌱 Classe: 0
   └── Direita:
      📌 Se newbalanceOrig <= 84478.06:
      ├── Esquerda:
         📌 Se o

In [30]:
def prever(no, exemplo):
    """Percorre a árvore de decisão e retorna a previsão para um exemplo."""
    if no.resultado is not None:
        return no.resultado  # Retorna a classe da folha

    # Segue para a esquerda ou direita baseado no valor da feature
    if exemplo[no.feature] <= no.valor_corte:
        return prever(no.esquerda, exemplo)
    else:
        return prever(no.direita, exemplo)

# Aplicando a previsão em todo o conjunto de teste
y_pred_polars = X_test.apply(lambda x: prever(arvore_decisao, x.to_dict()), axis=1)

# Avaliação do modelo manual
print("📊 Performance da Árvore de Decisão Manual")
print("Acurácia:", accuracy_score(y_test, y_pred_polars))
print("Relatório de Classificação:\n", classification_report(y_test, y_pred_polars))


📊 Performance da Árvore de Decisão Manual
Acurácia: 0.999030273692284
Relatório de Classificação:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00   1270904
           1       0.71      0.40      0.51      1620

    accuracy                           1.00   1272524
   macro avg       0.86      0.70      0.75   1272524
weighted avg       1.00      1.00      1.00   1272524



In [31]:
import time

# Testando a velocidade da árvore manual
start_time = time.time()
y_pred_polars = X_test.apply(lambda x: prever(arvore_decisao, x.to_dict()), axis=1)
tempo_manual = time.time() - start_time

# Testando a velocidade do Scikit-Learn
start_time = time.time()
y_pred_sklearn = modelo_sklearn.predict(X_test)
tempo_sklearn = time.time() - start_time

print(f"⏱ Tempo de inferência - Árvore Manual (Polars): {tempo_manual:.4f} segundos")
print(f"⏱ Tempo de inferência - Scikit-Learn: {tempo_sklearn:.4f} segundos")


⏱ Tempo de inferência - Árvore Manual (Polars): 13.6634 segundos
⏱ Tempo de inferência - Scikit-Learn: 0.0917 segundos
